In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

# 載入檔案

In [3]:
dataframe1 = pd.read_csv('001_win.csv')
dataframe2 = pd.read_csv('002_win.csv')
dataframe3 = pd.read_csv('003_win.csv')
dataframe = pd.concat([dataframe1, dataframe2, dataframe3], ignore_index= True , axis= 0 ) 
fi = pd.read_csv('finaaal.csv')
item = pd.read_csv('item.csv')

In [4]:
dataframe = pd.read_csv('003_win.csv')
item = pd.read_csv('item.csv')

In [5]:
df1 = dataframe.drop(['NO','性別','年齡' ,'月收入' ,'居住地' ,'職業', '學歷', '姓名', '地址', '電話'], axis = 1)
df_q = df1.iloc[:, list(range(0,202,2))]  
df_q.shape

(220, 101)

In [6]:
df_q.rename(columns={'q35 ':'q35'}, inplace=True)
df_q.rename(columns={'q66 ':'q66'}, inplace=True)

In [7]:
q = list(item.對應題目.unique())[0]
count = item[item['對應題目']== q].子題名稱.count()

In [8]:
def transdf(df, item):
    df2 = pd.DataFrame()
    for i in range(0,101):
        q = list(item.對應題目.unique())[i]
        count = item[item['對應題目']== q].子題名稱.count()
        o = list(item[item['對應題目']== q].index)
        df = df_q[[q]].replace(list(range(1,count-1))+[ 0, 99], list(item.iloc[o,3]))
        df2 = pd.concat([df2, df], ignore_index= True, axis= 1)
    df2.columns = list(item.調查品項.unique())
    return df2

In [9]:
df001 = transdf(df_q, item)

In [11]:
df001.head(3)

,行動電話系統業者,數據網路服務,房屋仲介,租車,快遞,加油站,連鎖咖啡店,連鎖藥妝店╱藥局,保全公司,直銷公司,...,拋棄式隱形眼鏡,隱形眼鏡藥水,維他命,感冒藥,嬰兒紙尿片╱褲,報紙,電視台,綜藝節目主持人,談話性節目主持人,政治人物
0,中華電信,中華電信,信義,未填答,順豐,中油,星巴克,杏一,未填答,美樂家,...,帝康TICON,博士倫ReNu,未填答,普拿疼（伏冒）,大王GOON,自由時報,三立,蔡康永,陳文茜,馬英九
1,中華電信,中華電信,未填答,和運(和泰),宅配通,中油,未填答,丁丁藥局,未填答,Amway安麗,...,未填答,未填答,善存Centrum,普拿疼（伏冒）,幫寶適Pampers(寶僑),自由時報,三立,胡瓜,趙少康,賴清德
2,中華電信,中華電信,永慶,未填答,黑貓宅急便（統一速達）,中油,星巴克,啄木鳥,未填答,Amway安麗,...,未填答,未填答,善存Centrum,克風邪,妙而舒Merries(花王),其他,民視,吳宗憲,鄭弘儀,賴清德
3,中華電信,中華電信,信義,未填答,UPS優比速,中油,星巴克,杏一,其他,NU SKIN,...,博士倫(舒服能),博士倫ReNu,萊萃美Nature Made,普拿疼（伏冒）,好奇Huggies(Kleenex),經濟日報,GTV八大,張小燕,鄭弘儀,馬英九
4,台灣大哥大,其他,台灣房屋,和運(和泰),黑貓宅急便（統一速達）,中油,星巴克,屈臣氏,未填答,Amway安麗,...,視康(愛爾康)CIBA Vision,愛爾康ALCON,克補,普拿疼（伏冒）,幫寶適Pampers(寶僑),蘋果日報,三立,陶晶瑩,劉寶傑,朱立倫


# 建立各行業別名稱名稱

In [12]:
#https://stackoverflow.com/questions/9232256/round-up-to-third-decimal-place-in-python
from math import ceil, floor
def float_round(num, places = 0, direction = floor):
    return direction(num * (10**places)) / float(10**places)

In [13]:
#inside.append('{}%'.format('%.1f' % (float_round(ind.iloc[j,0],3, ceil)*100/df_q.shape[0])))
#inside.append('{}%'.format('%.1f' % (float_round(ind.iloc[3,0], 2, ceil)*100/df_q.shape[0])))

In [14]:
inside = []
for i in range(0,101): 
    colname = df001.columns[i]    
    ind = pd.DataFrame(pd.value_counts(df001[colname].replace(['未填答','其他'],[np.NaN,np.NaN]).values, sort=True))
    for j in range(3):        
        inside.append(ind.index[j])
        inside.append('%.1f' % (float_round(ind.iloc[j,0],3, ceil)*100/df_q.shape[0]))
    if len(ind) > 4:
        if ind.iloc[3,0] == ind.iloc[2,0]: 
            inside.append('{}(並列第三名)'.format(ind.index[3]))
            inside.append('%.1f' % (float_round(ind.iloc[3,0], 2, ceil)*100/df_q.shape[0]))
        else:
            inside.append('無')
            inside.append('--')
    else:
        inside.append('無')
        inside.append('--')

In [16]:
x = np.array(inside)
x = x.reshape(101, 8)
final2 = pd.DataFrame(x, columns=['第一名名稱', '第一名%','第二名名稱', '第二名%','第三名名稱', '第三名%','備註名稱', '備註%'])

# 合併調查品項

In [17]:
final1 = pd.DataFrame()
final1['對應題目'] = list(df_q.columns)
final1['調查品項'] = ['行動電話系統業者','數據網路服務', '房屋仲介','租車', '快遞', '加油站', '連鎖咖啡店', '連鎖藥妝店╱藥局',
 '保全公司', '直銷公司', '建設公司', '航空公司', '金控集團', '人壽保險', '產物保險', '銀行', '旅行社', '大型賣場', '電池', '染髮劑',
 '洗髮精', '沐浴乳', '一般刮鬍刀', '電動刮鬍刀', '男性洗面乳', '女性洗面乳', '女性內衣', '衛生棉', '電視機（液晶）', '冷氣╱空調', '洗衣機', '冰箱',
 '電子鍋(電鍋)', '空氣清淨機', '微波爐', '桌上型電腦', '筆記型電腦', '顯示器（Monitor）', '數位相機', '手機', 'GPS（衛星導航）硬體', '行車記錄器', '多功能事務機',
 '投影機', '吸塵器', '連鎖傢俱', '衛浴設備', '熱水器', '瓦斯爐', '抽油煙機', '淨水器', '果菜汁╱果汁', '鮮奶', '優酪乳', '碳酸氣泡飲料','運動飲料',
 '罐裝咖啡', '即溶咖啡', '茶飲料', '包裝水', '啤酒', '白蘭地', '威士忌', '速食麵', '成人奶粉', '嬰幼兒奶粉', '喉糖', '冰淇淋', '喜餅', '包裝米', '醬油', '雞精', '洗衣粉╱精', '洗碗精', '牙膏', '面紙',
 '衛生紙', '機車', '自行車', '輪胎', '房車', '休旅車', '運動鞋', '休閒服', '男鞋', '女鞋', '男錶', '女錶', '按摩椅', '血壓計', '體脂計',
 '拋棄式隱形眼鏡', '隱形眼鏡藥水', '維他命', '感冒藥', '嬰兒紙尿片╱褲', '報紙', '電視台', '綜藝節目主持人', '談話性節目主持人', '政治人物']

In [18]:
final = pd.concat([final1, final2], ignore_index= False, axis= 1 ) 

In [19]:
final.shape

(101, 10)

In [20]:
for i in range(101):
    if final.iloc[i,9] != '--':
        final.iloc[i,9] = final.iloc[i,9] + '%'
    else :
        final.iloc[i,9] = final.iloc[i,9]

In [21]:
#final.head(5)

In [22]:
#final.to_csv('{}.csv'.format('中區-前三名'),encoding='utf-8')

# [xlsx](https://xlsxwriter.readthedocs.io/)

In [23]:
final[['第一名%', '第二名%', '第三名%']] = final[['第一名%', '第二名%', '第三名%']].astype('float')

In [24]:
six = final[final['第一名%']>=60].sort_values(by='第一名%').drop('對應題目', axis = 1)
fif_six = final[np.logical_and(final['第一名%']<60, final['第一名%']>=50)].sort_values(by='第一名%',ascending=0).drop('對應題目', axis = 1)
four_fif = final[np.logical_and(final['第一名%']<50, final['第一名%']>=40)].sort_values(by='第一名%',ascending=0).drop('對應題目', axis = 1)
third_four = final[np.logical_and(final['第一名%']<40, final['第一名%']>=30)].sort_values(by='第一名%',ascending=0).drop('對應題目', axis = 1)
twen_third = final[np.logical_and(final['第一名%']<30, final['第一名%']>=20)].sort_values(by='第一名%',ascending=0).drop('對應題目', axis = 1)
ten_twen = final[np.logical_and(final['第一名%']<20, final['第一名%']>=10)].sort_values(by='第一名%',ascending=0).drop('對應題目', axis = 1)

In [25]:
percentage = [six, fif_six, four_fif, third_four, twen_third, ten_twen]
for i in percentage:
    i[['第一名%', '第二名%', '第三名%']] = i[['第一名%', '第二名%', '第三名%']].astype(str) + '%'

# 心佔率存檔

In [26]:
step0 =1
step1 = 1 + six.shape[0] +3
step2 = step1 + fif_six.shape[0] + 3
step3 = step2 + four_fif.shape[0] + 3
step4 = step3 + third_four.shape[0] + 3
step5 = step4 + twen_third.shape[0] + 3
print(step0, step1, step2, step3, step4, step5)

1 6 13 27 48 90


In [27]:
writer = pd.ExcelWriter('心占率(南區).xlsx', engine='xlsxwriter') 

six.to_excel(writer, sheet_name='Sheet1', startrow= step0, index=False)  
fif_six.to_excel(writer, sheet_name='Sheet1', startrow= step1, index=False)
four_fif.to_excel(writer, sheet_name='Sheet1', startrow= step2, index=False)
third_four.to_excel(writer, sheet_name='Sheet1', startrow= step3 , index=False)
twen_third.to_excel(writer, sheet_name='Sheet1', startrow= step4, index=False)
ten_twen.to_excel(writer, sheet_name='Sheet1', startrow= step5, index=False)

writer.save()

In [28]:
import openpyxl as px
wb = px.load_workbook('心占率(南區).xlsx')
ws = wb.active
p = wb.get_sheet_by_name(name = 'Sheet1')
ws['A{}'.format(step0)] = '第一名 60%以上廠商及該項的第二、第三名的心占率'
ws['A{}'.format(step1)] = '第一名 50%-59%以上廠商及該項的第二、第三名的心占率'
ws['A{}'.format(step2)] = '第一名 40%-49以上廠商及該項的第二、第三名的心占率'
ws['A{}'.format(step3)] = '第一名 30%-39%以上廠商及該項的第二、第三名的心占率'
ws['A{}'.format(step4)] = '第一名 20%-29%以上廠商及該項的第二、第三名的心占率'
ws['A{}'.format(step5)] = '第一名 10%-19%以上廠商及該項的第二、第三名的心占率'
wb.save('心占率(南區).xlsx')

C:\Users\hahohy889\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  after removing the cwd from sys.path.


# 前三名存檔

In [29]:
final[['第一名%', '第二名%', '第三名%']] = final[['第一名%', '第二名%', '第三名%']].astype(str) + '%'

In [30]:
final.head(5)

,對應題目,調查品項,第一名名稱,第一名%,第二名名稱,第二名%,第三名名稱,第三名%,備註名稱,備註%
0,q01,行動電話系統業者,中華電信,45.9%,台灣大哥大,22.3%,遠傳電信,19.5%,無,--
1,q02,數據網路服務,中華電信,58.6%,速博(Seednet)\n/遠傳sparq寬頻,15.5%,亞太,2.7%,無,--
2,q03,房屋仲介,信義,28.6%,永慶,23.6%,住商,10.9%,無,--
3,q04,租車,和運(和泰),33.2%,格上(裕隆),27.7%,小馬,5.9%,無,--
4,q05,快遞,黑貓宅急便（統一速達）,55.0%,宅配通,11.4%,中華郵政,10.5%,無,--


In [31]:
#final.to_excel('{}.xlsx'.format('南區-前三名'),encoding='utf-8', index = False )

In [32]:
diction = {'第一名 60%以上廠商及該項的第二、第三名的心占率' : six,
           '第一名 50%-59%以上廠商及該項的第二、第三名的心占率': fif_six,
           '第一名 40%-49以上廠商及該項的第二、第三名的心占率': four_fif,
           '第一名 30%-39%以上廠商及該項的第二、第三名的心占率': third_four,
           '第一名 20%-29%以上廠商及該項的第二、第三名的心占率': twen_third,
           '第一名 10%-19%以上廠商及該項的第二、第三名的心占率': ten_twen}